In [1]:
from datasets import load_dataset
nsmc_dataset = load_dataset('nsmc')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to C:/Users/shsmc/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
nsmc_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [3]:
nsmc_dataset['train'][0]

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

In [4]:
nsmc_dataset['train'].features

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [6]:
nsmc_dataset['train'].features['label'].str2int('negative')

0

In [7]:
nsmc_df = nsmc_dataset['train'].to_pandas()

In [8]:
nsmc_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [10]:
nsmc_df.groupby('label').count()

,id,document
label,,
0,75173,75173
1,74827,74827


In [11]:
nsmc_df['review_length'] = nsmc_df['document'].str.len()
nsmc_df.review_length.describe()

count    150000.000000
mean         35.203353
std          29.532097
min           0.000000
25%          16.000000
50%          27.000000
75%          42.000000
max         146.000000
Name: review_length, dtype: float64

In [13]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

C:\Users\shsmc\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shsmc\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
tok

BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [15]:
tok.tokenize('청춘 영화의 최고봉.')

['청', '##춘', '영화', '##의', '최고', '##봉', '.']

In [16]:
tok('청춘 영화의 최고봉')

{'input_ids': [101, 9751, 97707, 42428, 10459, 83491, 118989, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tok(['청춘 영화의 최고봉.', '청춘'], padding=True)

{'input_ids': [[101, 9751, 97707, 42428, 10459, 83491, 118989, 119, 102], [101, 9751, 97707, 102, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0]]}

In [18]:
def tokenizer(data):
    return tok(data['document'], max_length=32, truncation=True, padding='max_length')

In [19]:
nsmc_dataset_tokenized = nsmc_dataset.map(tokenizer)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
nsmc_dataset_tokenized['train'][0]

{'id': '9976970',
 'document': '아 더빙.. 진짜 짜증나네요 목소리',
 'label': 0,
 'input_ids': [101,
  9519,
  9074,
  119005,
  119,
  119,
  9708,
  119235,
  9715,
  119230,
  16439,
  77884,
  48549,
  9284,
  22333,
  12692,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [21]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
num_train_epochs = 2
learning_rate = 2e-7
batch_size = 128

In [24]:
from torch.utils.data import DataLoader

tr_ds = nsmc_dataset_tokenized['train'].remove_columns(['id', 'document'])
tr_ds.set_format(type='torch')

In [25]:
tr_dl = DataLoader(tr_ds, batch_size=batch_size)


In [26]:
val_ds = nsmc_dataset_tokenized['test'].remove_columns(['id','document'])
val_ds.set_format(type='torch')
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [27]:
next(iter(tr_dl))

{'label': tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
         0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
         0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
         0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
         0, 0, 1, 1, 0, 1, 0, 1]),
 'input_ids': tensor([[  101,  9519,  9074,  ...,     0,     0,     0],
         [  101,   100,   119,  ..., 16439,   102,     0],
         [  101,   100,   102,  ...,     0,     0,     0],
         ...,
         [  101,  9358, 12508,  ...,     0,     0,     0],
         [  101,  9519, 25503,  ...,     0,     0,     0],
         [  101, 10150, 10954,  ...,  9568, 12310,   102]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 

In [28]:
import numpy as np
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

def acc(pred, label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item

In [32]:
device

device(type='cuda')

In [33]:
model = model.to(device)

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_train_epochs):
    train_losses = []
    train_accc = 0
    model.train()
    
    for step, batch in enumerate(tqdm(tr_dl)):
        label = batch['label'].to(device)
        input_id, token_type_ids, attention_mask = batch['input_ids'].to(device), batch['token_type_ids'].to(device), batch['attention_mask']
        
        model.zero_grad()
        pred = model(input_id, token_type_ids, attention_mask)
        
        loss = criterion(torch.sigmoid(pred.logits.t()[1]), label.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        train_acc += acc(pred.logits.argmax(dim=1), label)
        
    print("train loss:", np.mean(train_losses))
    print("train acc:", train_acc/len(tr_dl.dataset))
    
    model.eval()
    val_losses = []
    val_acc = 0
    
    for step, batch in enumerate(tqdm(val_dl)):
        label = batch['label'].to(device)
        input_id, token_type_ids, attention_mask = batch['input_ids'].to(device), batch['token_type_ids'].to(device), batch['attention_mask']
        
        pred = model(input_id, token_type_ids, attention_mask)
        
        loss = criterion(torch.sigmoid(pred.logits.t()[1]), label.float())
        
        val_losses.append(loss.item)
        val_ac += acc(pred.logits.argmax(dim=1), label)
        
    print("val_loss: ", np.mean(val_losses))
    print("val acc: ", val_accc/len(val_dl.dataset))

  0%|                                                                                         | 0/1172 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)